# Assignment 3. GFF feature files and visualization
## Gema Castillo García

### Files I have created:
The main script that makes the GFF files and the report is:

    make_files.rb
    
The GFF files are:

    gene_coordinates.gff
    chr_coordinates.gff

The report file with the genes without the repeat is:

    genes_without_rep.txt

The file with the GFF track beside the AT2G46340 gene on the ENSEMBL website is:

    Arabidopsis_thaliana_219021079_19028603.png


### Final program:
    $ ruby make_files.rb
    

**Your biologist collaborators are going to do a site-directed/insertional mutagenesis in Arabidopsis, using the list of 167 genes from your last assignment as the desired targets.  Inserts will be targeted to the repeat CTTCTT, and they want inserts to go into EXONS.**

### 1. Using BioRuby, examine the sequences of the ~167 Arabidopsis genes from the last assignment by retrieving them from whatever database you wish.

### 2. Loop over every exon feature, and scan it for the CTTCTT sequence.

### 3. Take the coordinates of every CTTCTT sequence and create a new Sequence Feature (you can name the feature type, and source type, whatever you wish; the start and end coordinates are the first ‘C’ and the last ‘T’ of the match.).  Add that new Feature to the EnsEMBL Sequence object.  (YOU NEED TO KNOW:  When you do regular expression matching in Ruby, use RegEx/MatchData objects; there are methods that will tell you the starting and ending coordinates of the match in the string).

In [1]:
##showing sequences, exons, positions, and more in the output of Jupyter's boxes to see if I am retrieving the information properly
##these outputs are not shown in the terminal when you run the .rb file because I have hidden those lines with '#'


require 'bio'
require 'rest-client'  

#creating a function called "fetch" that we can re-use everywhere in our code ()
def fetch(url, headers = {accept: "*/*"}, user = "", pass="") #fetch definition was written by Mark Wilkinson
  response = RestClient::Request.execute({
    method: :get,
    url: url.to_s,
    user: user,
    password: pass,
    headers: headers})
  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response  #now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response  #now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  #now we are returning 'False', and we will check that with an \"if\" statement in our main code
end


def read_file(file) #defining a function to read and store the content of "ArabidopsisSubNetwork_GeneList.txt"
  abort("Please, check the name of #{file}, I can't find it!\n") unless (File.exists?(file)) #aborting if the file does not exist
  puts "Reading the file...\n"
  @agi = File.read(file).split("\n") #reading the file
  return @agi
end


###EX 1. Using BioRuby, examine the sequences of the ~167 Arabidopsis genes from the last assignment by retrieving them from whatever database you wish.###  
def retrieve_seq(agi_list) #defining a function to obtain the sequences of the AGI codes and their positions for the target "CTTCTT"
  File.open("gene_coordinate.gff", "w") { |f| f.write("##gff-version 3\n") } #creating a GFF file for EX 4a
  File.open("chr_coordinate.gff", "w") { |f| f.write("##gff-version 3\n") } #creating a GFF file for EX 5

  fasta = {} #creating a hash for Bio::Sequence objects (key=AGI) - all the genes of the list -
  @repeats = {} #creating a hash for Bio::Sequence objects with Bio::Feature objects (key=AGI) - only the genes with repeats -
  if @agi.empty?
    puts "This file is empty!" #aborting if the file is empty
  else
    puts "Retrieving sequences from Ensembl with Dbfetch..."
    @agi.each do |geneid| #for each gene (geneid) of the file do...
      url = "https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&style=raw&id=#{geneid}" #retrieveing Ensembl information of 'geneid'
      res = fetch(url)
      if res  #res is either the response object, or False, so you can test it with 'if'
        body = res.body  #getting the "body" of the response
        if body.empty?
          puts "There is no entry for #{geneid} in Ensembl." #printing a friendly message if the gene has not an entry in Ensembl
        else
          embl = Bio::EMBL.new(res.body) #creating a Bio::EMBL object for 'geneid'
          puts "\n\n\n" + embl.sv #showing chromosomic coordinates of 'geneid' for EX 5
          fasta[geneid] = Bio::Sequence.new(embl.to_biosequence) #creating a Bio::Sequence::NA object for 'geneid'
          puts ">" + geneid + "\n" + fasta[geneid] #showing 'geneid' sequence in FASTA format
          
          
          #inserts will be targeted to the repeat "CTTCTT" on both the + and - strands of DNA
          rep_f = Bio::Sequence::NA.new("CTTCTT") #to search rep "CTTCTT" on + strand
          rep_f_re = /(?=(#{rep_f.to_s}))/ #this step allows to search for overlapping targets by tranforming 'rep_f' into a reg expression
          #as the EMBL's sequences are forward, I also have to search for the complementary sequence of that repeat, that is, "AAGAAG"
          rep_r = rep_f.reverse_complement #to search rep "CTTCTT" on - strand
          rep_r_re = /(?=(#{rep_r.to_s}))/ #this step allows to search for overlapping targets by tranforming 'rep_r' into a reg expression
          
          ###EX 2. Loop over every exon feature, and scan it for the "CTTCTT" sequence.###
          embl.features do |feature| #looking for the repetition "CTTCTT" in geneid's exons
            if feature.feature == "exon"
              if /:/.match(feature.position) #ignoring overlaped exons
                next
              elsif feature.position =~ /complement([^"]+)/ #searching for exons on - strand (complementary strands)
                puts "\t\nEXON in reverse strand:" + $1[1..$1.length-2] #showing gene coordinates of the exon
                pos1 = $1[1..$1.length-2]
                pos1_1 = pos1.match(/(?<start>\d+)..(?<end>\d+)/)
                pos1_2 = (pos1_1[:start].to_i..pos1_1[:end].to_i) #getting coordinates of the exon in BIO-informatics style (first nucleotide is '1')
                exon = fasta[geneid][pos1_2.begin-1..pos1_2.end-1] #changing coordinates to informatics style (first nucleotide is '0') to retrieve the exon's sequence  
                #puts exon
                positions_f = exon.enum_for(:scan, rep_f_re).map { Regexp.last_match.begin(0) } #getting coordinates of the forward repetitions inside the exon
                puts "\texon positions with #{rep_f.to_s}: #{positions_f}" unless positions_f.empty? #showing the exon coordinates of the forward repeat
                positions_r = exon.enum_for(:scan, rep_r_re).map { Regexp.last_match.begin(0) } #getting coordinates of the reverse repetitions inside the exon
                puts "\texon positions with #{rep_r.to_s}: #{positions_r}" unless positions_r.empty? #showing the exon coordinates of the reverse repeat
                
                
                
                if embl.sv =~ /chromosome:TAIR10:([^"]+):([^"]+):([^"]+):([^"]+)/ #searching for chromosomic coordinates for EX 5
                  chr = $1 #retrieving the chromosome
                  chr_pos = ($2.to_i..$3.to_i) #retrieving the chromosomic coordinates of the gene
                  
                  ###EX 3. Create a new Sequence Feature with the coordinates of every CTTCTT sequence and add it to the EnsEMBL Sequence object.### 
                  fasta[geneid].features = []  #creating an array to store the new feature for geneid's sequence (Bio::Sequence object)
                  positions_f.each do |f| ##creating a Bio::Feature object to store the information about the forward repeats
                    @feature = Bio::Feature.new('exon_rep',pos1_2,
                      [ Bio::Feature::Qualifier.new("cttctt", [pos1_2.select.with_index { |pos, idx| idx==f }[0], pos1_2.select.with_index { |pos, idx| idx==f+5 }[0]] ), 
                        Bio::Feature::Qualifier.new('strand', '+')
                        ])
                    fasta[geneid].features << @feature #adding the new Bio::Feature object to the Bio::Sequence object
                    puts "\t\tgene position #{pos1_2.select.with_index { |pos, idx| idx==f }}: #{@feature}" #showing the gene coordinates of the Bio::Feature object

                    ###EX 4. Loop over each one of your CTTCTT features and create a GFF3-formatted file of these features###
                    File.open("gene_coordinate.gff", "a") { |f| f.write("#{seqid=geneid}\t#{source="."}\t#{type="insertion_site"}\t#{start=@feature.qualifiers[0].value[0]}\t#{final=@feature.qualifiers[0].value[1]}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }   
                    
                    ###EX 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL###
                    File.open("chr_coordinate.gff", "a") { |f| f.write("#{seqid=chr}\t#{source="."}\t#{type="insertion_site"}\t#{start=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[0]-1 }}\t#{final=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[1]-1 }}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }  
                  end

                  positions_r.each do |r| ##creating a Bio::Feature object to store the information about the reverse repeats
                    @feature = Bio::Feature.new('exon_rep',pos1_2,
                      [ Bio::Feature::Qualifier.new("aagaag", [pos1_2.select.with_index { |pos, idx| idx==r }[0], pos1_2.select.with_index { |pos, idx| idx==r+5 }[0]] ),
                        Bio::Feature::Qualifier.new('strand', '-')
                        ])
                    fasta[geneid].features << @feature #adding the new Bio::Feature object to the Bio::Sequence object
                    puts "\t\tgene position #{pos1_2.select.with_index { |pos, idx| idx==r }}: #{@feature}" #showing the gene coordinates of the Bio::Feature object

                    ###EX 4. Loop over each one of your CTTCTT features and create a GFF3-formatted file of these features###
                    File.open("gene_coordinate.gff", "a") { |f| f.write("#{seqid=geneid}\t#{source="."}\t#{type="insertion_site"}\t#{start=@feature.qualifiers[0].value[0]}\t#{final=@feature.qualifiers[0].value[1]}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }   
                    ###EX 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL###
                    File.open("chr_coordinate.gff", "a") { |f| f.write("#{seqid=chr}\t#{source="."}\t#{type="insertion_site"}\t#{start=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[0]-1 }}\t#{final=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[1]-1 }}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }  
                  end
                end
                #puts fasta[geneid].features
                @repeats[geneid] = fasta[geneid]
                #puts @repeats[geneid]
                
              else #searching for exons on + strand
                puts "\t\nEXON in forward strand:" + feature.position #showing gene coordinates of the exon
                pos1 = feature.position
                pos1_1 = pos1.match(/(?<start>\d+)..(?<end>\d+)/)
                pos1_2 = (pos1_1[:start].to_i..pos1_1[:end].to_i) #getting coordinates of the exon in BIO-informatics style (first nucleotide is '1')
                exon = fasta[geneid][pos1_2.begin-1..pos1_2.end-1] #changing coordinates to informatics style (first nucleotide is '0') to retrieve the exon's sequence  
                puts exon #showing exon's sequence
                positions_f = exon.enum_for(:scan, rep_f_re).map { Regexp.last_match.begin(0) } #getting coordinates of "CTTCTT" repetitions inside the exon (not gene coordinates)     
                puts "\texon positions with #{rep_f.to_s}: #{positions_f}" unless positions_f.empty? #showing the exon coordinates of the forward repeat
                positions_r = exon.enum_for(:scan, rep_r_re).map { Regexp.last_match.begin(0) } #getting coordinates of "AAGAAG" repetitions inside the exon (not gene coordinates)   
                puts "\texon positions with #{rep_r.to_s}: #{positions_r}" unless positions_r.empty? #showing the exon coordinates of the reverse repeat
                
                
                
                if embl.sv =~ /chromosome:TAIR10:([^"]+):([^"]+):([^"]+):([^"]+)/ #searching for chromosomic coordinates for EX 5
                  chr = $1 #retrieving the chromosome
                  chr_pos = ($2.to_i..$3.to_i) #retrieving the chromosomic coordinates of the gene
                  
                  ###EX 3. Create a new Sequence Feature with the coordinates of every "CTTCTT" sequence and add it to the EnsEMBL Sequence object.### 
                  fasta[geneid].features = [] #creating an array to store the new feature for geneid's sequence (Bio::Sequence object)
                  positions_f.each do |f| ##creating a Bio::Feature object to store the information about the forward repeats
                    @feature = Bio::Feature.new('exon',pos1_2,
                      [ Bio::Feature::Qualifier.new("cttctt", [pos1_2.select.with_index { |pos, idx| idx==f }[0], pos1_2.select.with_index { |pos, idx| idx==f+5 }[0]] ), 
                        Bio::Feature::Qualifier.new('strand', '+')
                        ])
                    fasta[geneid].features << @feature #adding the new Bio::Feature object to the Bio::Sequence object
                    puts "\t\tgene position #{pos1_2.select.with_index { |pos, idx| idx==f }}: #{@feature}" #showing the gene coordinates of the Bio::Feature object

                    ###EX 4a. Loop over each one of your CTTCTT features and create a GFF3-formatted file of these features###
                    File.open("gene_coordinate.gff", "a") { |f| f.write("#{seqid=geneid}\t#{source="."}\t#{type="insertion_site"}\t#{start=@feature.qualifiers[0].value[0]}\t#{final=@feature.qualifiers[0].value[1]}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }   
                    ###EX 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL###
                    File.open("chr_coordinate.gff", "a") { |f| f.write("#{seqid=chr}\t#{source="."}\t#{type="insertion_site"}\t#{start=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[0]-1 }}\t#{final=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[1]-1 }}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }  
                  end
                  positions_r.each do |r| ##creating a Bio::Feature object to store the information about the reverse repeats
                    @feature = Bio::Feature.new('exon',pos1_2,
                      [ Bio::Feature::Qualifier.new("aagaag", [pos1_2.select.with_index { |pos, idx| idx==r }[0], pos1_2.select.with_index { |pos, idx| idx==r+5 }[0]] ),
                        Bio::Feature::Qualifier.new('strand', '-')
                        ])
                    fasta[geneid].features << @feature #adding the new Bio::Feature object to the Bio::Sequence object
                    puts "\t\tgene position #{pos1_2.select.with_index { |pos, idx| idx==r }}: #{@feature}" #showing the gene coordinates of the Bio::Feature object

                    ###EX 4a. Loop over each one of your CTTCTT features and create a GFF3-formatted file of these features###
                    File.open("gene_coordinate.gff", "a") { |f| f.write("#{seqid=geneid}\t#{source="."}\t#{type="insertion_site"}\t#{start=@feature.qualifiers[0].value[0]}\t#{final=@feature.qualifiers[0].value[1]}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }   
                    ###EX 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL###
                    File.open("chr_coordinate.gff", "a") { |f| f.write("#{seqid=chr}\t#{source="."}\t#{type="insertion_site"}\t#{start=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[0]-1 }}\t#{final=chr_pos.select.with_index { |pos, idx| idx==@feature.qualifiers[0].value[1]-1 }}\t#{score="."}\t#{strand=@feature.qualifiers[1].value}\t#{phase="."}\t#{attributes="ID=#{geneid}"};repeat=#{type=@feature.qualifiers[0].qualifier}\n") }  
                  end
                end
                #puts fasta[geneid].features
                @repeats[geneid] = fasta[geneid]
              end
            end
          end
        end
      end
    end
  end
  puts "\n\n\nDONE - now you can check your fasta sequences and exons with repeats above"
end


###EX 4a. Loop over each one of your CTTCTT features and create a GFF3-formatted file of these features###
def write_gene_gff_file(gfffile) #defining a function to delete the duplicated lines of the GFF file created with the previous function
  abort("Please, check the name of #{gfffile}, I can't find it!\n") unless (File.exists?(gfffile)) #aborting if the file does not exist
  gff = File.read(gfffile).split("\n")
  array = [] #creating an array to store the lines of the GFF file
  gff.each do |lines|
    array << lines
  end
  array = array.uniq #deleting duplicated lines

  array.each do |l|
    puts l
    File.open("gene_coordinates.gff", "a") { |f| f.write("#{l}\n") } #creating a new GFF file without the duplicated lines
  end

  File.delete(gfffile) #deleting the GFF file with duplicated lines
end


###EX 4b. Output a report showing which (if any) genes on your list do NOT have exons with the CTTCTT repeat###
def write_genes_wo_rep(agi_file,gff_file)
  abort("Please, check the name of #{agi_file}, I can't find it!\n") unless (File.exists?(agi_file)) #aborting if the agi_file does not exist
  abort("Please, check the name of #{gff_file}, I can't find it!\n") unless (File.exists?(gff_file)) #aborting if the gff_file does not exist
  gff_genes = [] #creating an array for the genes present in the GFF file
  gff = File.read(gff_file).split("\n") #reading the GFF file
  gff.each do |lines| #for each line of the GFF file do...
    gff_genes << lines[0..8] if lines =~ /A[Tt]\d[Gg]\d\d\d\d\d/ #retrieving the first 9 characters of each line if it contains an AGI code (i.e., ignoring the first line of the GFF file)
  end
  gff_genes = gff_genes.uniq #deleting duplicates of the same gene

  File.open("genes_without_rep.txt", "a") { |f| f.write("Genes without a CTTCTT repeat\n") } #creating a new file for the report with the genes without the "CTTCTT" repeat
  agi = File.read(agi_file).split #reading 'ArabidopsisSubNetwork_GeneList.txt'
  agi.each do |gene| #for each gene of the agi_file do...
    File.open("genes_without_rep.txt", "a") { |f| f.write("- #{gene}\n") } unless gff_genes.include? gene #adding the genes not present in the GFF file to the report with the genes without the "CTTCTT" repeat
  end
end


###EX 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL###
def write_chr_gff_file(chrfile) #defining a function to delete the duplicated lines of the GFF file created with the previous function
  abort("Please, check the name of #{chrfile}, I can't find it!\n") unless (File.exists?(chrfile)) #aborting if the file does not exist
  gff = File.read(chrfile).split("\n")
  array = [] #creating an array to store the lines of the GFF file
  gff.each do |lines|
    array << lines.tr('[]', '')
  end
  array = array.uniq #deleting duplicated lines
  
  array.each do |l|
    puts l
    File.open("chr_coordinates.gff", "a") { |f| f.write("#{l}\n") } #creating a new GFF file without the duplicated lines
  end
  
  File.delete(chrfile) #deleting the GFF file with duplicated lines
end

:write_chr_gff_file

In [2]:
read_file("ArabidopsisSubNetwork_GeneList.txt") #creating a variable with all the AGI codes of the list

Reading the file...



["AT4g27030", "AT5g54270", "AT1g21400", "AT5g19120", "AT2g13360", "AT4g05180", "AT1g22690", "AT2g45170", "AT4g09650", "AT5g55620", "AT1g31330", "AT1g80440", "AT3g28270", "AT4g12800", "AT5g04140", "AT5g64040", "AT1g29660", "AT1g15820", "AT1g64500", "AT1g03130", "AT2g20670", "AT4g17090", "AT1g23740", "AT2g46340", "AT4g17460", "AT1g07010", "AT1g32220", "AT2g03750", "AT3g47470", "AT4g12830", "AT5g05690", "AT5g64410", "AT2g21330", "AT1g15980", "AT2g01890", "AT1g12250", "AT2g21210", "AT4g28660", "AT1g52230", "AT3g18890", "AT4g22890", "AT1g11850", "AT1g37130", "AT2g04039", "AT3g48420", "AT4g19170", "AT5g07020", "AT5g65010", "AT3g01500", "AT1g25230", "AT2g34430", "AT1g18060", "AT2g30600", "AT4g33010", "AT1g65490", "AT3g21670", "AT4g27260", "AT1g12780", "AT1g55480", "AT2g17880", "AT3g49160", "AT4g21210", "AT5g14740", "AT4g37980", "AT1g29395", "AT2g39730", "AT1g20020", "AT2g34420", "AT4g37800", "AT1g70820", "AT3g26740", "AT4g28780", "AT1g12900", "AT1g64680", "AT2g25200", "AT3g55800", "AT4g21280"

In [3]:
retrieve_seq(@agi) #returns DNA sequences, exons, and more of "ArabidopsisSubNetwork_GeneList.txt" genes and creates a GFF file (use this GFF file as input for the following function)   

Retrieving sequences from Ensembl with Dbfetch...



chromosome:TAIR10:4:13571813:13573070:1
>AT4g27030
gtgctagtgatcagaatagccccacgcaacactaacgtacaccacaacataacttgagcgtaatctcgtcctctatcatgtggttatgccgtcgagaaaacgagaacatttgacaactttcacctgcaatcactctcaatggctgtatcacttccaaccaagtaccctctacgacctatcaccaacatcccaaaaagccaccgtccctcgcttctccgtgtacgtgtcacctgctctgttactaccaccaagcctcagcctaatcgtgagaagcttctggtagagcaacgcactgtgaatcttcctctgtccaacgaccaatctctgcaatcgaccaagcctcgccctaaccgtgagaagcttgtggttgagcaacgccttgccagccctcctctgtccaatgacccaactttgaaatcgacatggactcaccggttatgggttgcagcgggctgcaccactttgtttgtctctttagctaaatctgtcattggagggtttgattctcatctctgcctcgaaccagctttagccggttatgcagggtacatcttagctgatctaggttccggtgtctaccactgggccattgataactacggtgatgagtcaacacctgtagtaggaacccaaatcgaagcatttcagggtcaccacaagtggccttggacaatcaccagacggcaatttgccaacaatctacacgctctggctcaagtcataaccttcacagttcttccactagaccttgcatttaacgaccctgtgtttcacggctttgtgtgcacatttgcattttgcatattgtttagccagcaattccatgcttgggcacatggaaccaagagcaagcttccacctctcgtggtcgcgttgcaggacatggggttacttgt

	
EXON in forward strand:2861..3498
cttctgcaagcgattcaggctgcggagaagtgggagaaacaaccattgacagagttgtttaacgatgtatatgatgttaaaccgaagaacctagaagagcaagaacttggtttgaaggaattagtaaagaaacaacctcaagattatcctcctggctttcatgtttgaatctagaggaactgtgtggttaaaatacctcatatagtatcatgtccatatcgacttctcattgcagactatttatattgtccacgtatcgaatagtaatcaagtatcaatgtagagaccagcatttggagcatcatgaaatggccttgagaatgttattttcatggtccggctgacccttaaccggttcaaactggagtaaatgagtgaatgttaatgctcgagaatgggtgaggctccgttaatcgttaagaaaaaaacgtgcctacctagccaaatgacatcttcctcttttgtgtttagcacaagtgaatgatccaaatatcatctacctaatatgacccataaaatagtatgtgattgtctttggatgttatcccataacataacttaaatttgggatgcatgcatatattatatacttcatataaaatgagtggacctcataagttgcggtttcgatttttatc
	
EXON in forward strand:922..1369
tacaacaacaacaacagttcacatccgcacaagtcatggtaaaggtagggaccacgtcgcatgagaagtaaaagtggggacctcaagttacacgtatgaataatactaaatcaagtgctatatgggtcccacttgtgtagttgatttgagtacgtgtacgtctcattcattagtcaatagtcactgcagtctgaatcttccaaaaaacagacactaaatttcactatgatccctgtaattattaaactttcatatatgctcccaatatgattagttcttaacacattaaa

	exon positions with cttctt: [107, 110, 140, 148]
		gene position [214]: #<Bio::Feature:0x000055b6f8fa74e0>
		gene position [217]: #<Bio::Feature:0x000055b6f907a458>
		gene position [247]: #<Bio::Feature:0x000055b6f8fee8e0>
		gene position [255]: #<Bio::Feature:0x000055b6f8f20558>
	
EXON in forward strand:2589..2774
atgatgacatatagagtaggacatcattctacatcagatgattcaactaagtacagggcggcggatgaaatccagtactggaaaatgtcgagaaaccctgtgaatagatttcggaaatgggtcgaagataacggatggtggagtgaggaagatgaatccaagctaagatctaacgcaagaaaacag
	
EXON in forward strand:2861..3498
cttctgcaagcgattcaggctgcggagaagtgggagaaacaaccattgacagagttgtttaacgatgtatatgatgttaaaccgaagaacctagaagagcaagaacttggtttgaaggaattagtaaagaaacaacctcaagattatcctcctggctttcatgtttgaatctagaggaactgtgtggttaaaatacctcatatagtatcatgtccatatcgacttctcattgcagactatttatattgtccacgtatcgaatagtaatcaagtatcaatgtagagaccagcatttggagcatcatgaaatggccttgagaatgttattttcatggtccggctgacccttaaccggttcaaactggagtaaatgagtgaatgttaatgctcgagaatgggtgaggctccgttaatcgttaagaaaaaaacgtgcctacctagccaaatgacatcttcctc

	
EXON in reverse strand:2496..2740
	exon positions with cttctt: [70, 73]
		gene position [2566]: #<Bio::Feature:0x000055b6fa383d88>
		gene position [2569]: #<Bio::Feature:0x000055b6fa382000>
	
EXON in reverse strand:2496..2670
	exon positions with cttctt: [70, 73]
		gene position [2566]: #<Bio::Feature:0x000055b6fa386d30>
		gene position [2569]: #<Bio::Feature:0x000055b6fa384fa8>
	
EXON in reverse strand:1224..1590
	
EXON in reverse strand:1674..2196
	exon positions with cttctt: [331]
		gene position [2005]: #<Bio::Feature:0x000055b6fa388720>
	
EXON in reverse strand:2496..2616
	exon positions with cttctt: [70, 73]
		gene position [2566]: #<Bio::Feature:0x000055b6fa38d270>
		gene position [2569]: #<Bio::Feature:0x000055b6f9ec2920>
	
EXON in reverse strand:275..815
	
EXON in reverse strand:1674..2191
	exon positions with cttctt: [331]
		gene position [2005]: #<Bio::Feature:0x000055b6f9ebd4c0>
	
EXON in reverse strand:501..815
	
EXON in reverse strand:894..1104
	
EXON in reverse strand:

	exon positions with cttctt: [206, 254, 330]
		gene position [1250]: #<Bio::Feature:0x000055b6f9bad000>
		gene position [1298]: #<Bio::Feature:0x000055b6fa7668b0>
		gene position [1374]: #<Bio::Feature:0x000055b6fa764b28>
	
EXON in forward strand:444..499
aaaagagaaaagctgaagctggaaggatcagggagaaataccctgatcgaattcct
	
EXON in forward strand:1..115
atccaatcatagacgaaaagaaaaaggttcctttttttgactttgtatccgtagatcatcttcttcttcttcttccagagttttatccttatccgttccatcaaattctctctct
	exon positions with cttctt: [59, 62, 65, 68]
		gene position [60]: #<Bio::Feature:0x000055b6f9b9dc40>
		gene position [63]: #<Bio::Feature:0x000055b6fa77b4b8>
		gene position [66]: #<Bio::Feature:0x000055b6fa779730>
		gene position [69]: #<Bio::Feature:0x000055b6f9b97110>
	
EXON in forward strand:1044..1528
gtaccttgtgccatcagacctaacagttggtcaatttgtgtatgtgattcggaagagaatcaaactaagtgcagagaaagctatcttcattttcgtagacaatgttcttcctccaacaggagagctaatgtcaagcgtttacgaggataagaaagacgaagatggcttcctttacatcacttacagtggcgagaacacattcggtgcttcttcaatctaatctctgacc

...

	exon positions with cttctt: [144, 184, 479]
		gene position [802]: #<Bio::Feature:0x000055b6fd99fa20>
		gene position [842]: #<Bio::Feature:0x000055b6fd99dc98>
		gene position [1137]: #<Bio::Feature:0x000055b6fc6e3e40>
	
EXON in forward strand:1..414
aaaagaaaactgaagaaccccaaaaatcccaagtgaaaacgttatccacttcgagagagtgacacgtcacgaagcaaatccaaattcaaaacctttccataaaaatcaaatcacttcttcttattacttcccccaaatcgaaatcagtcacttaaaaccctaaattatgacgatcgcaccggcattgcagacgacgttcgtgtcatcaaccaacttcctgaaacattcttcttcgtggggatcaccatcaccgaacaatgtgattcttcccaaaaacaagagatcttcttcctccgtagttgtcgccgccgtcggtgatgtctcttctgacggaacaatctacttaatcggcggagccatcgccgttgcactcgtcggaactgcattcccgatcctcttcaaacgcaaagacac
	exon positions with cttctt: [113, 116, 227, 284]
		gene position [114]: #<Bio::Feature:0x000055b6fd9a4f70>
		gene position [117]: #<Bio::Feature:0x000055b6fc6d7078>
		gene position [228]: #<Bio::Feature:0x000055b6fc6d4f58>
		gene position [285]: #<Bio::Feature:0x000055b6fd9aaf60>



chromosome:TAIR10:2:2033154:2034283:1
>AT2g05540

	
EXON in forward strand:1..1802
aaatcactggcaaataggcagtcttgtcccagtaatggaacccacacgacatagcttcaaaaaaaaaacaaggaacttcctcatttcttctatttatattactcaaatctaaatcatccctaagaaatagtaatacacaaaaagaaacaaaatggtgaccttaacttcatcttcttcaactccaaacgtatcttttgatttcatgatgaacaataataataacagtaacaatctctatggtcctttctcttcttcttctacttctttctcttacttgacaagcaaggaagatgctctcacccagaagaacctaatgagtggtatcaccaatgatgttcttggaattaacaagaaagcttctgaagatttggagatcagtgtgtttggagctgagaagtacttcaatggagatatggattctgaccacagtcctagacttgtgtctcctctgccagatccagaagttccaatagagagaatctttgttggtccgaagcaaagctcgaagaattcctcggaaactcctagtcttcggtctgaatccagttggaatagccagagtttgttgcttcagagcaaatatgtagagaagaagaagaacatcaagaaaaactctagttgcaacagttactttcaggagaaggatatgagtagcaatcacaaggtgagtaataagaagagttttctcgctactcttggctgcagatgcgtgtgttcgaactggagttcagtggatgtcgtcgacgacaagagaagaagctctggtctgaagaagatcaagactcaattgagtttctctggagatctaagctcggagatgaagattcatcaacaacaacaagaagcaatgctagagcagaggaagtctctggaaatatttggatctcctcttattgagaagagaatcattcagaagaaatttccatgggaatactcgagctctgcgaaaaaagaagaacatggattc

	
exon positions with aagaag: [11, 14]
		gene position [1858]: #<Bio::Feature:0x000055b6fdee7500>
		gene position [1861]: #<Bio::Feature:0x000055b6fdee5778>
	
EXON in reverse strand:1666..1767
	
EXON in reverse strand:1666..2150
	
exon positions with aagaag: [110, 192, 195]
		gene position [1776]: #<Bio::Feature:0x000055b6fdeed3d8>
		gene position [1858]: #<Bio::Feature:0x000055b6fc2bdf00>
		gene position [1861]: #<Bio::Feature:0x000055b6fdef6b68>
	
EXON in reverse strand:1137..1177
	
EXON in reverse strand:106..216
	
EXON in reverse strand:731..837
	
EXON in reverse strand:302..436
	exon positions with cttctt: [31]
		gene position [333]: #<Bio::Feature:0x000055b6fc2b5288>
	
EXON in forward strand:1495..1688
tgaaaccgttgcctattaattcctaacaagagagatcatcactgccaagaactcatcaaatgtgacatagagaggacaacttcaagaagatgtagccacagactatacattacaatatctcatttgtaccaatgtgtatgtaatttttgatcgagtcaagtgttaaggtaccattccgactgcagtgccggcga
	exon positions with aagaag: [83]
		gene position [1578]: #<Bio::Feature:0x000055b6fdf1

	exon positions with aagaag: [394]
		gene position [998]: #<Bio::Feature:0x000055b6fe110ac0>
	
EXON in forward strand:1..226
catttctgtgatcgccaaagccacacgatgattcttgcctaaatcattttaaagactgtatagaggaaaacaaaactgcaaaaacaaaaataaaaaaaacatcgcacaagaaaataaaagatttgtagaatcaactaagaaaatggctagcactatgatgactacattgcctcagttcaatggtcttcgagccaccaaaatctctgcagctcctgtacaaggcctg



chromosome:TAIR10:1:29518184:29519323:1
>AT1g78460
ttttgatcaggtgctttcttttacagctttataagtgttctagaaagcgaaactgaaaaaaaaaaaaagcgaaactgaaaaaatcatttacagtttttttttgattcgatgatatttaggtttgattttcaattttgattcaataatcttagttattatttaatcttttatcaaactttttggttacagttttccccacctttacaaaataacaatattaatcaccacttaaccagagatacaatttataccggttaaaccagaagctgcaagctgctgttacagtatcgatcggttctaaaccacctcttctaatccaattgctacgtaacaatcaaatcgtgagggcgtcattaacatgatctacatgttaaacggaaccatgatctcattaatcaccctagccgtggtatgagatggcgggttataaacagcgagaaaatacgccggcaactttcctttgctcttctcgatgggcaaaatccagttagaatcccggaggctttccatcagagccgcagcttcgtccttggctacattgttcgacacgtagccaccgatctgtctcacggctacgtaagttggtttcgtaga

	
EXON in forward strand:1..728
aatggacttgaaacatcaccagataacgcagagtttaatagataagattagtaatcctactcaccgtcagatggatttgtgaatcgatccaacggtcataatcgtctttccaagataggccagctccgaaactcacctccacgcatctataatcgtgtatcaagaaacggatctttcttctctatcttctgaatatcacaaaacccaattagccggagatggccgcaacatccgccgctgctgcagctgcttcttccatcatgggtactcgggtggctcccggtatccatcccggttcaggtcggttcacagccgtgttcggtttcggaaagaagaaggcagctcccaaaaagagtgccaaaaagacggtgactacggaccggcctctttggtacccaggcgccatttctcctgactggcttgatggttccttggttggagattacggttttgatcccttcggtttaggcaaaccggccgagtatcttcaattcgatatcgattcactagaccagaatctggctaagaacttggccggagacgtgatcggaacccgtacggaagctgccgacgccaaatcgacgccgtttcagccgtacagtgaggtgttcggaatccagagattcagggaatgcgaactcatccacggacggtgggcgatgctcgctactctcggcgctctctccgtcgaatggcttaccggcgttacatggcaagacgctggcaag
	exon positions with cttctt: [249]
	exon positions with aagaag: [329, 332]
		gene position [250]: #<Bio::Feature:0x000055b6fe709e90>
		gene position [330]: #<Bio::Feature:0x000055b6fe708108>
		gene position [333]: #<Bio::Feature:0x000055b6

### 4a. Once you have found them all, and added them all, loop over each one of your CTTCTT features (using the #features method of the EnsEMBL Sequence object) and create a GFF3-formatted file of these features.

In [4]:
write_gene_gff_file("gene_coordinate.gff") #creates a GFF file without duplicated lines and with gene coordinates of the repeat "CTTCTT" 

##gff-version 3
AT5g54270	.	insertion_site	208	213	.	-	.	ID=AT5g54270;repeat=aagaag
AT5g54270	.	insertion_site	1099	1104	.	+	.	ID=AT5g54270;repeat=cttctt
AT5g54270	.	insertion_site	1311	1316	.	-	.	ID=AT5g54270;repeat=aagaag
AT1g21400	.	insertion_site	2004	2009	.	-	.	ID=AT1g21400;repeat=aagaag
AT1g21400	.	insertion_site	214	219	.	+	.	ID=AT1g21400;repeat=cttctt
AT1g21400	.	insertion_site	217	222	.	+	.	ID=AT1g21400;repeat=cttctt
AT1g21400	.	insertion_site	247	252	.	+	.	ID=AT1g21400;repeat=cttctt
AT1g21400	.	insertion_site	255	260	.	+	.	ID=AT1g21400;repeat=cttctt
AT1g21400	.	insertion_site	102	107	.	-	.	ID=AT1g21400;repeat=aagaag
AT1g21400	.	insertion_site	1572	1577	.	-	.	ID=AT1g21400;repeat=aagaag
AT5g19120	.	insertion_site	102	107	.	+	.	ID=AT5g19120;repeat=cttctt
AT5g19120	.	insertion_site	105	110	.	+	.	ID=AT5g19120;repeat=cttctt
AT5g19120	.	insertion_site	108	113	.	+	.	ID=AT5g19120;repeat=cttctt
AT5g19120	.	insertion_site	124	129	.	+	.	ID=AT5g19120;repeat=cttctt
AT5g19120	.	insertion_si

AT1g07010	.	insertion_site	2455	2460	.	-	.	ID=AT1g07010;repeat=aagaag
AT1g32220	.	insertion_site	1678	1683	.	+	.	ID=AT1g32220;repeat=cttctt
AT1g32220	.	insertion_site	1710	1715	.	-	.	ID=AT1g32220;repeat=aagaag
AT1g32220	.	insertion_site	1050	1055	.	+	.	ID=AT1g32220;repeat=cttctt
AT1g32220	.	insertion_site	1349	1354	.	+	.	ID=AT1g32220;repeat=cttctt
AT2g03750	.	insertion_site	419	424	.	+	.	ID=AT2g03750;repeat=cttctt
AT2g03750	.	insertion_site	422	427	.	+	.	ID=AT2g03750;repeat=cttctt
AT2g03750	.	insertion_site	1172	1177	.	+	.	ID=AT2g03750;repeat=cttctt
AT2g03750	.	insertion_site	81	86	.	-	.	ID=AT2g03750;repeat=aagaag
AT2g03750	.	insertion_site	965	970	.	-	.	ID=AT2g03750;repeat=aagaag
AT3g47470	.	insertion_site	1479	1484	.	-	.	ID=AT3g47470;repeat=aagaag
AT4g12830	.	insertion_site	2120	2125	.	-	.	ID=AT4g12830;repeat=aagaag
AT4g12830	.	insertion_site	1666	1671	.	-	.	ID=AT4g12830;repeat=aagaag
AT5g05690	.	insertion_site	260	265	.	+	.	ID=AT5g05690;repeat=cttctt
AT5g05690	.	insertion_site	263	2

AT3g21670	.	insertion_site	745	750	.	+	.	ID=AT3g21670;repeat=cttctt
AT3g21670	.	insertion_site	965	970	.	+	.	ID=AT3g21670;repeat=cttctt
AT3g21670	.	insertion_site	1400	1405	.	+	.	ID=AT3g21670;repeat=cttctt
AT3g21670	.	insertion_site	322	327	.	-	.	ID=AT3g21670;repeat=aagaag
AT3g21670	.	insertion_site	1378	1383	.	-	.	ID=AT3g21670;repeat=aagaag
AT4g27260	.	insertion_site	1256	1261	.	-	.	ID=AT4g27260;repeat=aagaag
AT1g12780	.	insertion_site	188	193	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g12780	.	insertion_site	2013	2018	.	+	.	ID=AT1g12780;repeat=cttctt
AT1g12780	.	insertion_site	2587	2592	.	+	.	ID=AT1g12780;repeat=cttctt
AT1g12780	.	insertion_site	2200	2205	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g12780	.	insertion_site	2203	2208	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g12780	.	insertion_site	2219	2224	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g12780	.	insertion_site	2288	2293	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g12780	.	insertion_site	2365	2370	.	-	.	ID=AT1g12780;repeat=aagaag
AT1g55480	.	insertion_site	1

AT4g21280	.	insertion_site	933	938	.	+	.	ID=AT4g21280;repeat=cttctt
AT4g21280	.	insertion_site	989	994	.	-	.	ID=AT4g21280;repeat=aagaag
AT5g15850	.	insertion_site	291	296	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	381	386	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	384	389	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	990	995	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	993	998	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	1657	1662	.	+	.	ID=AT5g15850;repeat=cttctt
AT5g15850	.	insertion_site	1449	1454	.	-	.	ID=AT5g15850;repeat=aagaag
AT5g15850	.	insertion_site	611	616	.	-	.	ID=AT5g15850;repeat=aagaag
AT1g06680	.	insertion_site	698	703	.	+	.	ID=AT1g06680;repeat=cttctt
AT1g06680	.	insertion_site	849	854	.	-	.	ID=AT1g06680;repeat=aagaag
AT1g06680	.	insertion_site	1534	1539	.	+	.	ID=AT1g06680;repeat=cttctt
AT1g06680	.	insertion_site	1771	1776	.	-	.	ID=AT1g06680;repeat=aagaag
AT1g06680	.	insertion_site	89	94	.	-	.	I

AT5g24210	.	insertion_site	1954	1959	.	+	.	ID=AT5g24210;repeat=cttctt
AT5g24210	.	insertion_site	650	655	.	-	.	ID=AT5g24210;repeat=aagaag
AT1g71030	.	insertion_site	847	852	.	-	.	ID=AT1g71030;repeat=aagaag
AT1g71030	.	insertion_site	1360	1365	.	-	.	ID=AT1g71030;repeat=aagaag
AT1g71030	.	insertion_site	344	349	.	-	.	ID=AT1g71030;repeat=aagaag
AT1g71030	.	insertion_site	415	420	.	-	.	ID=AT1g71030;repeat=aagaag
AT1g71030	.	insertion_site	544	549	.	-	.	ID=AT1g71030;repeat=aagaag
AT3g54050	.	insertion_site	151	156	.	+	.	ID=AT3g54050;repeat=cttctt
AT3g54050	.	insertion_site	334	339	.	-	.	ID=AT3g54050;repeat=aagaag
AT3g54050	.	insertion_site	535	540	.	-	.	ID=AT3g54050;repeat=aagaag
AT3g54050	.	insertion_site	1259	1264	.	-	.	ID=AT3g54050;repeat=aagaag
AT3g54890	.	insertion_site	471	476	.	+	.	ID=AT3g54890;repeat=cttctt
AT3g54890	.	insertion_site	522	527	.	+	.	ID=AT3g54890;repeat=cttctt
AT3g54890	.	insertion_site	525	530	.	+	.	ID=AT3g54890;repeat=cttctt
AT3g54890	.	insertion_site	1356	1361	.	-	.

AT1g69870	.	insertion_site	3506	3511	.	-	.	ID=AT1g69870;repeat=aagaag
AT1g69870	.	insertion_site	3515	3520	.	-	.	ID=AT1g69870;repeat=aagaag
AT1g69870	.	insertion_site	4338	4343	.	-	.	ID=AT1g69870;repeat=aagaag
AT1g69870	.	insertion_site	3189	3194	.	-	.	ID=AT1g69870;repeat=aagaag
AT2g47930	.	insertion_site	112	117	.	-	.	ID=AT2g47930;repeat=aagaag
AT2g47930	.	insertion_site	148	153	.	-	.	ID=AT2g47930;repeat=aagaag
AT2g47930	.	insertion_site	287	292	.	-	.	ID=AT2g47930;repeat=aagaag
AT2g47930	.	insertion_site	331	336	.	-	.	ID=AT2g47930;repeat=aagaag
AT4g02530	.	insertion_site	136	141	.	-	.	ID=AT4g02530;repeat=aagaag
AT4g02530	.	insertion_site	14	19	.	-	.	ID=AT4g02530;repeat=aagaag
AT4g02530	.	insertion_site	1722	1727	.	+	.	ID=AT4g02530;repeat=cttctt
AT4g02530	.	insertion_site	1788	1793	.	+	.	ID=AT4g02530;repeat=cttctt
AT4g02530	.	insertion_site	1853	1858	.	+	.	ID=AT4g02530;repeat=cttctt
AT4g38840	.	insertion_site	415	420	.	-	.	ID=AT4g38840;repeat=aagaag
AT4g38840	.	insertion_site	418	423	.

AT4g12310	.	insertion_site	1737	1742	.	+	.	ID=AT4g12310;repeat=cttctt
AT4g12310	.	insertion_site	1805	1810	.	+	.	ID=AT4g12310;repeat=cttctt
AT4g12310	.	insertion_site	1876	1881	.	+	.	ID=AT4g12310;repeat=cttctt
AT4g12310	.	insertion_site	953	958	.	+	.	ID=AT4g12310;repeat=cttctt
AT5g01530	.	insertion_site	250	255	.	+	.	ID=AT5g01530;repeat=cttctt
AT5g01530	.	insertion_site	330	335	.	-	.	ID=AT5g01530;repeat=aagaag
AT5g01530	.	insertion_site	333	338	.	-	.	ID=AT5g01530;repeat=aagaag
AT5g01530	.	insertion_site	1261	1266	.	+	.	ID=AT5g01530;repeat=cttctt
AT5g58260	.	insertion_site	1241	1246	.	+	.	ID=AT5g58260;repeat=cttctt
AT5g58260	.	insertion_site	1244	1249	.	+	.	ID=AT5g58260;repeat=cttctt
AT5g58260	.	insertion_site	1294	1299	.	+	.	ID=AT5g58260;repeat=cttctt
AT5g58260	.	insertion_site	1380	1385	.	+	.	ID=AT5g58260;repeat=cttctt


1

### 4b. Output a report showing which (if any) genes on your list do NOT have exons with the CTTCTT repeat.

In [5]:
write_genes_wo_rep("ArabidopsisSubNetwork_GeneList.txt","gene_coordinates.gff") #creates a txt file with the genes without "CTTCTT" repeat

["AT4g27030", "AT5g54270", "AT1g21400", "AT5g19120", "AT2g13360", "AT4g05180", "AT1g22690", "AT2g45170", "AT4g09650", "AT5g55620", "AT1g31330", "AT1g80440", "AT3g28270", "AT4g12800", "AT5g04140", "AT5g64040", "AT1g29660", "AT1g15820", "AT1g64500", "AT1g03130", "AT2g20670", "AT4g17090", "AT1g23740", "AT2g46340", "AT4g17460", "AT1g07010", "AT1g32220", "AT2g03750", "AT3g47470", "AT4g12830", "AT5g05690", "AT5g64410", "AT2g21330", "AT1g15980", "AT2g01890", "AT1g12250", "AT2g21210", "AT4g28660", "AT1g52230", "AT3g18890", "AT4g22890", "AT1g11850", "AT1g37130", "AT2g04039", "AT3g48420", "AT4g19170", "AT5g07020", "AT5g65010", "AT3g01500", "AT1g25230", "AT2g34430", "AT1g18060", "AT2g30600", "AT4g33010", "AT1g65490", "AT3g21670", "AT4g27260", "AT1g12780", "AT1g55480", "AT2g17880", "AT3g49160", "AT4g21210", "AT5g14740", "AT4g37980", "AT1g29395", "AT2g39730", "AT1g20020", "AT2g34420", "AT4g37800", "AT1g70820", "AT3g26740", "AT4g28780", "AT1g12900", "AT1g64680", "AT2g25200", "AT3g55800", "AT4g21280"

### 5. Re-execute your GFF file creation so that the CTTCTT regions are now in the full chromosome coordinates used by EnsEMBL.  Save this as a separate file - I will grade you on both 4a and 5 so be sure to submit both!

In [6]:
write_chr_gff_file("chr_coordinate.gff") #creates a GFF file without duplicated lines and with chromosomic coordinates of the repeat "CTTCTT"

##gff-version 3
5	.	insertion_site	22038372	22038377	.	-	.	ID=AT5g54270;repeat=aagaag
5	.	insertion_site	22039263	22039268	.	+	.	ID=AT5g54270;repeat=cttctt
5	.	insertion_site	22039475	22039480	.	-	.	ID=AT5g54270;repeat=aagaag
1	.	insertion_site	7495216	7495221	.	-	.	ID=AT1g21400;repeat=aagaag
1	.	insertion_site	7493426	7493431	.	+	.	ID=AT1g21400;repeat=cttctt
1	.	insertion_site	7493429	7493434	.	+	.	ID=AT1g21400;repeat=cttctt
1	.	insertion_site	7493459	7493464	.	+	.	ID=AT1g21400;repeat=cttctt
1	.	insertion_site	7493467	7493472	.	+	.	ID=AT1g21400;repeat=cttctt
1	.	insertion_site	7493314	7493319	.	-	.	ID=AT1g21400;repeat=aagaag
1	.	insertion_site	7494784	7494789	.	-	.	ID=AT1g21400;repeat=aagaag
5	.	insertion_site	6414589	6414594	.	+	.	ID=AT5g19120;repeat=cttctt
5	.	insertion_site	6414592	6414597	.	+	.	ID=AT5g19120;repeat=cttctt
5	.	insertion_site	6414595	6414600	.	+	.	ID=AT5g19120;repeat=cttctt
5	.	insertion_site	6414611	6414616	.	+	.	ID=AT5g19120;repeat=cttctt
5	.	insertion_site	6414614

1	.	insertion_site	2153910	2153915	.	-	.	ID=AT1g07010;repeat=aagaag
1	.	insertion_site	2155252	2155257	.	-	.	ID=AT1g07010;repeat=aagaag
1	.	insertion_site	11609531	11609536	.	+	.	ID=AT1g32220;repeat=cttctt
1	.	insertion_site	11609563	11609568	.	-	.	ID=AT1g32220;repeat=aagaag
1	.	insertion_site	11608903	11608908	.	+	.	ID=AT1g32220;repeat=cttctt
1	.	insertion_site	11609202	11609207	.	+	.	ID=AT1g32220;repeat=cttctt
2	.	insertion_site	1148192	1148197	.	+	.	ID=AT2g03750;repeat=cttctt
2	.	insertion_site	1148195	1148200	.	+	.	ID=AT2g03750;repeat=cttctt
2	.	insertion_site	1148945	1148950	.	+	.	ID=AT2g03750;repeat=cttctt
2	.	insertion_site	1147854	1147859	.	-	.	ID=AT2g03750;repeat=aagaag
2	.	insertion_site	1148738	1148743	.	-	.	ID=AT2g03750;repeat=aagaag
3	.	insertion_site	17494821	17494826	.	-	.	ID=AT3g47470;repeat=aagaag
4	.	insertion_site	7533260	7533265	.	-	.	ID=AT4g12830;repeat=aagaag
4	.	insertion_site	7532806	7532811	.	-	.	ID=AT4g12830;repeat=aagaag
5	.	insertion_site	1702899	1702904	.	+

3	.	insertion_site	7627501	7627506	.	+	.	ID=AT3g21670;repeat=cttctt
3	.	insertion_site	7627721	7627726	.	+	.	ID=AT3g21670;repeat=cttctt
3	.	insertion_site	7628156	7628161	.	+	.	ID=AT3g21670;repeat=cttctt
3	.	insertion_site	7627078	7627083	.	-	.	ID=AT3g21670;repeat=aagaag
3	.	insertion_site	7628134	7628139	.	-	.	ID=AT3g21670;repeat=aagaag
4	.	insertion_site	13654834	13654839	.	-	.	ID=AT4g27260;repeat=aagaag
1	.	insertion_site	4356113	4356118	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	4357938	4357943	.	+	.	ID=AT1g12780;repeat=cttctt
1	.	insertion_site	4358512	4358517	.	+	.	ID=AT1g12780;repeat=cttctt
1	.	insertion_site	4358125	4358130	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	4358128	4358133	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	4358144	4358149	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	4358213	4358218	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	4358290	4358295	.	-	.	ID=AT1g12780;repeat=aagaag
1	.	insertion_site	20715088	20715093	.	-	.	ID=

4	.	insertion_site	11335284	11335289	.	+	.	ID=AT4g21280;repeat=cttctt
4	.	insertion_site	11335340	11335345	.	-	.	ID=AT4g21280;repeat=aagaag
5	.	insertion_site	5176381	5176386	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5176471	5176476	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5176474	5176479	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5177080	5177085	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5177083	5177088	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5177747	5177752	.	+	.	ID=AT5g15850;repeat=cttctt
5	.	insertion_site	5177539	5177544	.	-	.	ID=AT5g15850;repeat=aagaag
5	.	insertion_site	5176701	5176706	.	-	.	ID=AT5g15850;repeat=aagaag
1	.	insertion_site	2048331	2048336	.	+	.	ID=AT1g06680;repeat=cttctt
1	.	insertion_site	2048482	2048487	.	-	.	ID=AT1g06680;repeat=aagaag
1	.	insertion_site	2049167	2049172	.	+	.	ID=AT1g06680;repeat=cttctt
1	.	insertion_site	2049404	2049409	.	-	.	ID=AT1g06680;repeat=aagaag
1	.	insertion_site	2047722	2047727	.	-	.	ID=

4	.	insertion_site	14202962	14202967	.	+	.	ID=AT4g28750;repeat=cttctt
5	.	insertion_site	8219144	8219149	.	+	.	ID=AT5g24210;repeat=cttctt
5	.	insertion_site	8217840	8217845	.	-	.	ID=AT5g24210;repeat=aagaag
1	.	insertion_site	26795898	26795903	.	-	.	ID=AT1g71030;repeat=aagaag
1	.	insertion_site	26796411	26796416	.	-	.	ID=AT1g71030;repeat=aagaag
1	.	insertion_site	26795395	26795400	.	-	.	ID=AT1g71030;repeat=aagaag
1	.	insertion_site	26795466	26795471	.	-	.	ID=AT1g71030;repeat=aagaag
1	.	insertion_site	26795595	26795600	.	-	.	ID=AT1g71030;repeat=aagaag
3	.	insertion_site	20016987	20016992	.	+	.	ID=AT3g54050;repeat=cttctt
3	.	insertion_site	20017170	20017175	.	-	.	ID=AT3g54050;repeat=aagaag
3	.	insertion_site	20017371	20017376	.	-	.	ID=AT3g54050;repeat=aagaag
3	.	insertion_site	20018095	20018100	.	-	.	ID=AT3g54050;repeat=aagaag
3	.	insertion_site	20339974	20339979	.	+	.	ID=AT3g54890;repeat=cttctt
3	.	insertion_site	20340025	20340030	.	+	.	ID=AT3g54890;repeat=cttctt
3	.	insertion_site	20340

1	.	insertion_site	26316236	26316241	.	+	.	ID=AT1g69870;repeat=cttctt
1	.	insertion_site	26319292	26319297	.	-	.	ID=AT1g69870;repeat=aagaag
1	.	insertion_site	26319301	26319306	.	-	.	ID=AT1g69870;repeat=aagaag
1	.	insertion_site	26320124	26320129	.	-	.	ID=AT1g69870;repeat=aagaag
1	.	insertion_site	26318975	26318980	.	-	.	ID=AT1g69870;repeat=aagaag
2	.	insertion_site	19617086	19617091	.	-	.	ID=AT2g47930;repeat=aagaag
2	.	insertion_site	19617122	19617127	.	-	.	ID=AT2g47930;repeat=aagaag
2	.	insertion_site	19617261	19617266	.	-	.	ID=AT2g47930;repeat=aagaag
2	.	insertion_site	19617305	19617310	.	-	.	ID=AT2g47930;repeat=aagaag
4	.	insertion_site	1112217	1112222	.	-	.	ID=AT4g02530;repeat=aagaag
4	.	insertion_site	1112095	1112100	.	-	.	ID=AT4g02530;repeat=aagaag
4	.	insertion_site	1113803	1113808	.	+	.	ID=AT4g02530;repeat=cttctt
4	.	insertion_site	1113869	1113874	.	+	.	ID=AT4g02530;repeat=cttctt
4	.	insertion_site	1113934	1113939	.	+	.	ID=AT4g02530;repeat=cttctt
4	.	insertion_site	18125392	18

3	.	insertion_site	5469508	5469513	.	+	.	ID=AT3g16140;repeat=cttctt
4	.	insertion_site	7311634	7311639	.	+	.	ID=AT4g12310;repeat=cttctt
4	.	insertion_site	7311702	7311707	.	+	.	ID=AT4g12310;repeat=cttctt
4	.	insertion_site	7311773	7311778	.	+	.	ID=AT4g12310;repeat=cttctt
4	.	insertion_site	7310850	7310855	.	+	.	ID=AT4g12310;repeat=cttctt
5	.	insertion_site	209115	209120	.	+	.	ID=AT5g01530;repeat=cttctt
5	.	insertion_site	209195	209200	.	-	.	ID=AT5g01530;repeat=aagaag
5	.	insertion_site	209198	209203	.	-	.	ID=AT5g01530;repeat=aagaag
5	.	insertion_site	210126	210131	.	+	.	ID=AT5g01530;repeat=cttctt
5	.	insertion_site	23561808	23561813	.	+	.	ID=AT5g58260;repeat=cttctt
5	.	insertion_site	23561811	23561816	.	+	.	ID=AT5g58260;repeat=cttctt
5	.	insertion_site	23561861	23561866	.	+	.	ID=AT5g58260;repeat=cttctt
5	.	insertion_site	23561947	23561952	.	+	.	ID=AT5g58260;repeat=cttctt


1

### 6. Prove that your GFF file is correct by uploading it to ENSEMBL and adding it as a new “track” to the genome browser of Arabidopsis (see http://plants.ensembl.org/info/website/upload/index.html. Along with your code, for this assignment please submit a screen-shot of your GFF track beside the **AT2G46340** gene on the ENSEMBL website to show me that you were successful.

See "Arabidopsis_thaliana_219021079_19028603.png".